We consider numerical results generated from ED and DMRG for the n-particle entanglement entropy of the $J$-$V$ model at half filling in the Luttinger Liquid phase $-2 < V/J < 2$.\begin{align}
H &= -J\sum_{i=1}^L(c_i^\dagger c_{i+1}+c^\dagger_{i+1}c_i)+V\sum_{i=1}^Ln_in_{i+1}
\end{align}
All the data files containing the coefficients from the fits can be generated from this notebook. The other notebook in this folder `Generating_Figures` can be used to generate the plots in the associated paper.

## 0. Importing packages and loading data functions

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.pyplot import cm
from functools import partial
import scipy.special as sp
from scipy.optimize import curve_fit
import os

### Loading data

In [2]:
filenames=os.listdir('../data/ed')
filenamesdmrg = os.listdir('../data/dmrg')

In [3]:
try:
    filenames.remove('.ipynb_checkpoints')
    filenamesdmrg.remove('.ipynb_checkpoints')
except:
    pass

In [4]:
def loaddata(α,v,ln): #used for C coefficients/fixed n
    N=[]
    oneN=[]
    ent=[] 
    for i in range(len(filenames)):    
        fn = filenames[i]
        n = int(fn.split("_")[2][1:])
        Nval = int(fn.split("_")[4][1:])
        if Nval > 10 and n == ln:
            with open('../data/ed/'+fn,"r",encoding="utf-8",errors='ignore') as file:
                lines = file.readlines()
                nLines = len(lines)
            data = np.loadtxt('../data/ed/'+fn,encoding="utf-8")    
            try:
                V  = data[:,0]
                S1 = data[:,α]
            except IndexError:
                V = np.array([data[0]])
                S1 = np.array([data[α]])
            if v in V:
                N.append(Nval)
                oneN.append(1/Nval)
                idx = np.where(V==v)
                ent.append(np.exp(-S1[idx]))
    for i in range(len(filenamesdmrg)):    
        fn = filenamesdmrg[i]
        n = int(fn.split("_")[2][1:])
        Nval = int(fn.split("_")[4][1:])
        if Nval > 10 and n==ln:
            with open('../data/dmrg/'+fn,"r",encoding="utf-8",errors='ignore') as file:
                lines = file.readlines()
                nLines = len(lines)
            data = np.loadtxt('../data/dmrg/'+fn,encoding="utf-8")    
            try:
                V  = data[:,0]
                S1 = data[:,α]
            except IndexError:
                V = np.array([data[0]])
                S1 = np.array([data[α]])
            if v in V:
                N.append(Nval)
                oneN.append(1/Nval)
                idx = np.where(V==v)
                ent.append(np.exp(-S1[idx]))
    return N, oneN,ent
def loaddatafull(α,v): #used for full data set/A coefficients
    nN=[]
    N=[]
    ent=[] 
    for i in range(len(filenames)):    
        fn = filenames[i]
        n = int(fn.split("_")[2][1:])
        Nval = int(fn.split("_")[4][1:])
        if Nval > 10:
            with open('../data/ed/'+fn,"r",encoding="utf-8",errors='ignore') as file:
                lines = file.readlines()
                nLines = len(lines)
            data = np.loadtxt('../data/ed/'+fn,encoding="utf-8")    
            try:
                V  = data[:,0]
                S1 = data[:,α]
            except IndexError:
                V = np.array([data[0]])
                S1 = np.array([data[α]])
            if v in V:
                nN.append(n/Nval)
                N.append(Nval)
                idx = np.where(V==v)
                ent.append(S1[idx])
    for i in range(len(filenamesdmrg)):    
        fn = filenamesdmrg[i]
        n = int(fn.split("_")[2][1:])
        Nval = int(fn.split("_")[4][1:])
        if Nval > 10:
            with open('../data/dmrg/'+fn,"r",encoding="utf-8",errors='ignore') as file:
                lines = file.readlines()
                nLines = len(lines)
            data = np.loadtxt('../data/dmrg/'+fn,encoding="utf-8")    
            try:
                V  = data[:,0]
                S1 = data[:,α]
            except IndexError:
                V = np.array([data[0]])
                S1 = np.array([data[α]])
            if v in V:
                ratio = n/Nval
                nN.append(ratio)
                N.append(Nval)
                idx = np.where(V==v)
                ent.append(S1[idx])
    return N, nN, ent
def addData(α,v, Nvalues, Svalues): #used for fixed ratio fitting
    for i in range(len(filenames)):    
        fn = filenames[i]
        n = int(fn.split("_")[2][1:])
        N = int(fn.split("_")[4][1:])
        key=getkey(n,N)
        if key != False:
            with open('../data/ed/'+fn,"r",encoding="utf-8",errors='ignore') as file:
                lines = file.readlines()
                nLines = len(lines)
            data = np.loadtxt('../data/ed/'+fn,encoding="utf-8")    
            try:
                V  = data[:,0]
                S = data[:,α]
            except IndexError:
                V = np.array([data[0]])
                S = np.array([data[α]])
            if v in V:
                idx = np.where(V==v)
                Nvalues[key].append(N)
                Svalues[key].append(S[idx])
    for i in range(len(filenamesdmrg)):    
        fn = filenamesdmrg[i]
        n = int(fn.split("_")[2][1:])
        N = int(fn.split("_")[4][1:])
        key=getkey(n,N)
        if key != False:
            with open('../data/dmrg/'+fn,"r",encoding="utf-8",errors='ignore') as file:
                lines = file.readlines()
                nLines = len(lines)
            data = np.loadtxt('../data/dmrg/'+fn,encoding="utf-8")    
            try:
                V  = data[:,0]
                S = data[:,α]
            except IndexError:
                V = np.array([data[0]])
                S = np.array([data[α]])
            if v in V:
                if N not in Nvalues[key]:
                    idx = np.where(V==v)
                    Nvalues[key].append(N)
                    Svalues[key].append(S[idx])
def getkey(n,N): #used for fixed ratio fitting
    ratio=N/n
    if ratio in ratios:
        key = f'{int(ratio)}'
        return key
    else:
        return False

## 1. Fitting Functions Used

In [5]:
def scalingform(x,c1,c2,c3,c4,c5,v): #fitting form for n=1, all other interactions
    exp = 4*g(v)
    return c1*x**4+c2*x**(3+exp)+c3*x**2+c4*x**(1+exp)+c5
def fitfunc(x,a,b,c): #fitting form for n=1, V/J=-1.65
    return a*x**4+b*x**2+c
def fitfunc23(x,a,b,c,d,e): #fitting form for n=2,3
    return a*x**4+b*x**3+c*x**2+d*x+e
def fitfuncfixed(x,a,b): #fitting form when ratios are fixed
    return a*x+b
def sinefit(ratio,A,B,C,D): #fitting form with different powers of sine with full data set
    x=np.sin(np.pi*ratio)
    return A*x**4+B*x**3+C*x**2+D*x

## 2. $C_{i}$ Coefficients

### Coefficients for $n=1$

In [8]:
nval=1
v=-1.65
data = []
for α in [1,2,3]:
    N, oneN, ent = loaddata(α,v,nval)
    oneN=np.array(oneN)
    ent = np.array(ent)
    ent=ent.reshape((-1,))
    fit, cov= curve_fit(fitfunc, oneN, ent)
    perr = np.sqrt(np.diag(cov))
    data.append((f'α',fit[2],perr[2],fit[1],perr[1],fit[0],perr[0]))
np.savetxt(f'../data/coefficients/CoeffsFixedn/Fixedncoefficientsv={v}.dat',data,header=r'α,$C_4$,$C_2$,$C_0$',encoding='utf-8',dtype=str)
vs = [-1.00,1.65,1.75]
#for v in vs:
    

TypeError: _savetxt_dispatcher() got an unexpected keyword argument 'dtype'

### Coefficients for $n=2,3$